In [1]:
import numpy as np
import pandas as pd
import math
import statistics
import matplotlib.pyplot as plt
from transformers import pipeline
from sklearn.model_selection import train_test_split, cross_val_score,KFold, cross_val_predict, GridSearchCV,RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier


In [2]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

## Download All Policies - Train Set

In [3]:
##Privacy Policies download:
websites = ['Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson']

sequence_to_classify = []

for i in websites:
    try:
        with open((i+".txt"), "r") as f:
            i = f.readlines()
            sequence_to_classify.append(i)
    except:
        with open((i+".txt"), "r", encoding='cp1252') as f:
            i = f.readlines()
            sequence_to_classify.append(i)

### Train Set Vectors:

In [4]:
policy_change_train = [1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,1,0,0,1,1,0,1,0,0,1,0,1,1,1,1,1,1,1,0,1,0,1,0,1,1,1,1,1,0,0]

### Privacy Policy Parameter

In [5]:
##Privacy Policy Change Matrix:

Phrases = ['Privacy Policy'
           #,'We may change this privacy policy at any time by posting an updated privacy policy on this Site'
           #,'Users are notified prior to any material changes to the policies'
           ,'We may update this Privacy Policy from time to time and without prior notice to you'
           #,'We will not make changes to this Privacy Statement or reduce the rights of the Users under this Privacy Statement without providing a notice thereof'
           #,'We may change this Privacy Statement from time to time. If we do, we will post the revised version here and change the “last updated date” at the top of the statement'
           #,'Please check our Privacy Policy periodically for changes'
           #,'We reserve the right to modify this privacy statement at any time, so please review it frequently'
           ,'we will update on privacy policy change'
           ,'you will be notify for policy changes'
           ,'we will tell you on privacy policy change'
           ,'we will not update you on privacy policy change'
           ,'in case of privacy policy change you will be notify'
           ,'in case of privacy policy change you will not be notify'
           #,'when we update our Privacy Policy, we will inform you'
          ]

##Matrix creation:
matrix_policy_change_ML = pd.DataFrame(columns = Phrases)
matrix_policy_change_ML['Privacy Policy'] = websites
matrix_policy_change_ML.set_index('Privacy Policy', inplace=True)

##Filling the matrix:
line = 0
for j in sequence_to_classify:
    for i in matrix_policy_change_ML:
        matrix_policy_change_ML.loc[websites[line],i] = classifier(str(j), str(i))["scores"][0]
    line += 1
    
matrix_policy_change_ML

,We may update this Privacy Policy from time to time and without prior notice to you,we will update on privacy policy change,you will be notify for policy changes,we will tell you on privacy policy change,we will not update you on privacy policy change,in case of privacy policy change you will be notify,in case of privacy policy change you will not be notify
Privacy Policy,,,,,,,
Google,0.988057,0.976609,0.944327,0.963079,0.079987,0.988186,0.919934
Aws,0.945912,0.908822,0.817005,0.853486,0.55011,0.887778,0.692566
AliExpress,0.917305,0.906227,0.832049,0.851518,0.131311,0.859369,0.405894
Meta,0.977616,0.97467,0.942786,0.935502,0.424396,0.966673,0.921531
TikTok,0.953713,0.935863,0.848715,0.867124,0.46969,0.888667,0.730353
YouTube,0.962129,0.968881,0.943104,0.918983,0.085857,0.972012,0.860169
Waze,0.986728,0.983069,0.931426,0.968472,0.33481,0.954483,0.819034
Wix,0.946877,0.928035,0.859711,0.814785,0.093084,0.938993,0.825347
Bookings,0.99324,0.982989,0.958652,0.964925,0.120652,0.986092,0.948679


### Test The Model - Privacy Policy

In [6]:
# X_train, X_test, y_train, y_test = train_test_split(matrix_policy_change, policy_change_train
#                                                     , test_size=0.5, random_state=42)

# Manual division so we will have all options (1/0) at both groups
X_train = matrix_policy_change_ML.iloc[:35,:] 
X_test = matrix_policy_change_ML.iloc[34:,:]
y_train = policy_change_train[:35]
y_test = policy_change_train[34:]

##Decission Tree Classifaier
regressor_tree = DecisionTreeClassifier()
regressor_tree = regressor_tree.fit(X_train, y_train)
pred_tree = regressor_tree.predict(X_test)
print('Decission Tree Classifaier:', pred_tree)
print(classification_report(y_test, pred_tree))

##Random Forest Classifaier
regressor_forest = RandomForestClassifier()
regressor_forest = regressor_forest.fit(X_train, y_train)
pred_forest = regressor_forest.predict(X_test).round()
print('Random Forest Classifaier:', pred_forest)
print(classification_report(y_test, pred_forest))

##Logistic Reg Classifier
clf = LogisticRegression()
clf.fit(X_train, y_train)
pred_clf = clf.predict(X_test)
print('Logistic Reg Classifier:', pred_clf)
print(classification_report(y_test, pred_clf))

##SVM
SVM = SVC(kernel='linear')
SVM = SVM.fit(X_train, y_train)
pred_SVM = SVM.predict(X_test)
print('SVM Classifier:', pred_SVM)
print(classification_report(y_test, pred_SVM))

##KNN
KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
KNN = KNN.fit(X_train, y_train)
pred_KNN = KNN.predict(X_test)
print('KNN Classifier:', pred_KNN)
print(classification_report(y_test, pred_KNN))

##neural_network
neural_network = MLPClassifier(random_state=1, max_iter=300)
neural_network = neural_network.fit(X_train, y_train)
pred_neural_network = neural_network.predict(X_test)
print('neural_network Classifier:', pred_neural_network)
print(classification_report(y_test, pred_neural_network))

Decission Tree Classifaier: [1 1 1 0 1 0 1 1 1 1 1 1 1 1 0]
              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.83      1.00      0.91        10

    accuracy                           0.87        15
   macro avg       0.92      0.80      0.83        15
weighted avg       0.89      0.87      0.86        15

Random Forest Classifaier: [1 1 1 1 1 1 1 1 1 1 0 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.64      0.90      0.75        10

    accuracy                           0.60        15
   macro avg       0.32      0.45      0.38        15
weighted avg       0.43      0.60      0.50        15

Logistic Reg Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.67      1.00      0.80        10

    ac

/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalb

neural_network Classifier: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.67      1.00      0.80        10

    accuracy                           0.67        15
   macro avg       0.33      0.50      0.40        15
weighted avg       0.44      0.67      0.53        15



/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/yuvalba/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [7]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_policy_change_Comparison = pd.DataFrame(columns = ['Phrase',
                                               'Google','Aws','AliExpress','Meta','TikTok','YouTube','Waze','Wix','Bookings','whatsapp'
            ,'apple','wolt','Visa','Mastercard','AirBNB','uber','Spotify','samsung','Wordpress','instagram'
            ,'MacDonalds','FDA','Oracle','Zara','cocacola','Xiaomi','Nasdaq','Walmart'
           ,'AirCanada','Lufthansa','shopify','Netflix','adobe','Starbucks','Shoppers','Decathlon','waltdisney'
            ,'AmericanEagle','lululemon','SAP','JetBrains','MySQLCode','Cadens','EpicGames'
            ,'unitedHealthGroup','Slack','SalesForce','JPMorgan','JohnsonAndJohnson'])
matrix_policy_change_Comparison['Phrase'] = ['Privacy Policy Change']

##Filling the matrix:
websites_index = 0
line = 0
pred = 0
policy_change_train_pred_Comparison = []
c = ['We update this Privacy Policy without prior notice','you will be notify for policy changes']

for j in sequence_to_classify:
    a = classifier(str(j),c)
    if a["labels"][0]==c[0] and a["scores"][0]>0.8:
        pred = 0
    else:
        pred = 1  
    matrix_policy_change_Comparison[websites[websites_index]].loc[line] = pred
    policy_change_train_pred_Comparison.append(pred)
    websites_index += 1
    pred = 0

matrix_policy_change_Comparison


,Phrase,Google,Aws,AliExpress,Meta,TikTok,YouTube,Waze,Wix,Bookings,...,SAP,JetBrains,MySQLCode,Cadens,EpicGames,unitedHealthGroup,Slack,SalesForce,JPMorgan,JohnsonAndJohnson
0,Privacy Policy Change,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [8]:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in policy_change_train:
    if policy_change_train_pred_Comparison[index]==1 and policy_change_train[index]==1:
        TP +=1
    elif policy_change_train_pred_Comparison[index]==0 and policy_change_train[index]==0:
        TN +=1
    elif policy_change_train_pred_Comparison[index]==0 and policy_change_train[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1

recall = TP/(TP+FN)
specificity = TN/(TN+FP)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("specificity:" , specificity)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 1.0
specificity: 0.0
Precision: 0.7346938775510204
Accuracy: 0.7346938775510204
F1 Score: 0.8470588235294118


In [9]:
##Privacy Policy Change Matrix:

##Matrix creation:
matrix_policy_change_Median = matrix_policy_change_ML.transpose()
    
#add the avg and median rows:
matrix_policy_change_Median.loc['Median'] =  matrix_policy_change_Median.median()
matrix_policy_change_Median.loc['Median']['Privacy Policy'] = 'Median'

policy_change_train_pred_Median = []
websites_index = 0

for i in matrix_policy_change_Median:
    num = matrix_policy_change_Median[websites[websites_index]].loc['Median']
    if num >0.84:
        policy_change_train_pred_Median.append(1)
    else:
        policy_change_train_pred_Median.append(0)
    websites_index += 1
 
print(policy_change_train_pred_Median)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1]


In [10]:
index = 0
TP = 0
FP = 0
FN = 0
TN = 0
for i in policy_change_train:
    if policy_change_train_pred_Median[index]==1 and policy_change_train[index]==1:
        TP +=1
    elif policy_change_train_pred_Median[index]==0 and policy_change_train[index]==0:
        TN +=1
    elif policy_change_train_pred_Median[index]==0 and policy_change_train[index]==1:
        FN +=1
    else:
        FP +=1
    index +=1

recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Accuracy = (TP+TN)/(TP+FP+FN+TN) 
F1 = (TP)/(TP+(0.5*(FP+FN)))

print("recall:" , recall)
print("Precision:" , Precision)
print("Accuracy:" , Accuracy)
print("F1 Score:" , F1)

recall: 1.0
Precision: 0.75
Accuracy: 0.7551020408163265
F1 Score: 0.8571428571428571
